Get a list of videos in your YouTube History and their transcripts and save them locally.

### Requirements

In [ ]:
! pip install requests
! pip install pandas
! pip install pyarrow
! pip install youtube_transcript_api

### Imports

In [38]:
import requests
import json
import pandas as pd
from datetime import datetime
from youtube_transcript_api import YouTubeTranscriptApi

### Implementation

Google API does not allow Histry data to be exported.  
So, I used [Google Takeout](https://takeout.google.com/) to download the YouTube History in a JSON file.  
Select only YouTube History and JSON format.  
Open the watch-history.json, clean-up ads from records, clean-up titles, select a date to filter out

In [60]:
# Read JSON data from a file
with open('watch-history.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
# Filter out records with 'details' field (to clean-up ads)
filtered_data = [record for record in data if 'details' not in record and record.get('titleUrl')]

# Convert to DataFrame
df = pd.DataFrame(filtered_data)

# Select required columns
df = df[['title', 'titleUrl', 'time']]

# Filter out rows where 'title' contains 'https' (to clean-up deleted videos)
df = df[~df['title'].str.contains('https')]

# Function to convert time to a uniform format
def convert_time(time_str):
    try:
        return datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    except ValueError:
        return datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%SZ")
# Convert 'time' to datetime
df['time'] = df['time'].apply(convert_time)
# Filter out rows where 'time' is before '2024-04-17' (my cut-off point that I want to search in)
df = df[df['time'] >= '2024-04-17']

# Remove 'Watched ' and various bad characters from the titles
replacements = {'Watched ':'',"—":"-","’":"'","‘":"'","“":"'","”":"'","â€”":'-','â€™':"'",'â€œ':"'",'!â€':"'"}
for key,value in replacements.items():
    df['title'] = df['title'].str.replace(key, value)

# Rename 'time' to 'datetime'
df = df.rename(columns={'time': 'datetime'})

# Keep only the most recent unique 'titleUrl'
df = df.sort_values('datetime', ascending=False).drop_duplicates('titleUrl', keep='first')

In [61]:
# Write to Parquet and CSV files
df.to_parquet('output.parquet', index=False)
df.to_csv('output.csv', index=False)

Get the transcripts of the videos in your History and add it to the data. On my PC it took 5m30s for 450 records.  

In [62]:
# Read the Parquet file
df = pd.read_parquet('output.parquet')

# Function to get transcript
def get_transcript(url):
    video_id = url.split('=')[-1]
    try:
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        transcript = ' '.join([i['text'] for i in transcript_list])
    except:
        transcript = None
    return transcript

# Apply the function to the 'titleUrl' column to create a new 'transcript' column
df['transcript'] = df['titleUrl'].apply(get_transcript)


In [63]:
# Write back to Parquet file
df.to_parquet('output_with_transcripts.parquet', index=False)
# CSV file has some problems with line breaks in the 'transcript' column. Parquet file works fine.
df.to_csv('output_with_transcripts.csv', index=False)